In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [2]:

url = "http://www.roadrun.co.kr/schedule/list.php"
response = requests.get(url)
response.encoding = 'euc-kr'  # 또는 'euc-kr' 시도 (확인 필요)

In [3]:
soup = BeautifulSoup(response.text, 'html.parser')


In [4]:
target_table = soup.select('body > p > table > tr:nth-of-type(3) >td')


In [5]:
p = target_table[0].find('p', attrs={'align': 'left'})

In [36]:
trs = p.select('table > tr > td > table:nth-of-type(3) >tr')


In [37]:
trs[0].find_all('td')[0].get_text(strip=True).split('(')[0].strip()

'5/3'

In [ ]:
for tr in trs:
    try: 
        tds = tr.find_all('td')
        # print(tds)
        if len(tds) < 4:
            continue
    except:
        print("예외발생")

In [40]:
race_data = []

In [41]:

from datetime import datetime


for tr in trs:
    try:
        tds = tr.find_all('td')
        if len(tds) < 4:
            continue  # 필드가 부족하면 스킵

        # 날짜 추출 및 포맷 변경
        date_raw = tds[0].get_text(strip=True).split('(')[0].strip()  # "5/3"
        # date_raw = tds[0].get_text()
        try:
            date_obj = datetime.strptime(date_raw, "%m/%d")
            race_date = f"2025-{date_obj.month:02}-{date_obj.day:02}"  # 연도는 임의 설정
        except:
            continue  # 날짜 형식이 이상하면 건너뜀
        
        # 접수중 여부 확인 (접수중이면 <img> 있음)
        is_open = bool(tds[0].find('img'))

        # 대회명
        race_title = tds[1].find('a').get_text(strip=True)

        # 코스 타입
        course_font = tds[1].find('font', attrs={'color': '#990000'})
        course_type = course_font.get_text(strip=True) if course_font else ''

        # 지역
        location = tds[2].get_text(strip=True)
        
        # 홈페이지 url
        home_url = ''
        for a in tds[3].find_all('a', href=True):
            if a['href'].startswith('http'):
                home_url = a['href']
                break

        race_data.append({
            "대회제목": race_title,
            "대회날짜": race_date,
            "지역": location,
            "코스타입": course_type,
            "접수중여부": "접수중" if is_open else "",
            "홈페이지": home_url
        })

    except Exception as e:
        # 디버깅 필요하면 print(e)
        continue

In [42]:
race_data

[{'대회제목': '제25회 여성마라톤',
  '대회날짜': '2025-05-03',
  '지역': '평화의공원 평화광장',
  '코스타입': '10km,5km,3km',
  '접수중여부': '',
  '홈페이지': 'http://www.womenmarathon.co.kr/'},
 {'대회제목': '2025 서울 유아차 런',
  '대회날짜': '2025-05-03',
  '지역': '광화문광장',
  '코스타입': '5km',
  '접수중여부': '',
  '홈페이지': 'http://seoulstrollerrun.co.kr/apply/guide.php'},
 {'대회제목': '제20회 보성 녹차 마라톤',
  '대회날짜': '2025-05-03',
  '지역': '보성공설운동장',
  '코스타입': '풀,하프,10km,5km',
  '접수중여부': '',
  '홈페이지': 'http://boseong.run1080.com/index.php?sub=sub01_m01_c01'},
 {'대회제목': '2025 우리은행배 제물포르네상스 국제 마라톤',
  '대회날짜': '2025-05-04',
  '지역': '인천 상상플랫폼',
  '코스타입': '10km,5km',
  '접수중여부': '',
  '홈페이지': 'http://run.ito.or.kr'},
 {'대회제목': '2025 한강 하프 마라톤',
  '대회날짜': '2025-05-04',
  '지역': '상암월드컵공원 평화광장',
  '코스타입': '하프,10km,5km',
  '접수중여부': '',
  '홈페이지': 'http://www.seoulhalfrun.kr/'},
 {'대회제목': '제23회 불,수,사,도,북5개산 종주',
  '대회날짜': '2025-05-04',
  '지역': '원자력병원.백세문',
  '코스타입': '42km,13km',
  '접수중여부': '',
  '홈페이지': 'http://koreatrail.net/home/main.php?link=csno&csno=201706131

In [45]:
df = pd.DataFrame(race_data)
print(df.head(5))

                        대회제목        대회날짜            지역           코스타입 접수중여부  \
0                 제25회 여성마라톤  2025-05-03    평화의공원 평화광장   10km,5km,3km         
1              2025 서울 유아차 런  2025-05-03         광화문광장            5km         
2             제20회 보성 녹차 마라톤  2025-05-03       보성공설운동장  풀,하프,10km,5km         
3  2025 우리은행배 제물포르네상스 국제 마라톤  2025-05-04      인천 상상플랫폼       10km,5km         
4             2025 한강 하프 마라톤  2025-05-04  상암월드컵공원 평화광장    하프,10km,5km         

                                                홈페이지  
0                    http://www.womenmarathon.co.kr/  
1      http://seoulstrollerrun.co.kr/apply/guide.php  
2  http://boseong.run1080.com/index.php?sub=sub01...  
3                               http://run.ito.or.kr  
4                        http://www.seoulhalfrun.kr/  


In [47]:
df.to_csv("marathon_schedule_2025.csv", index=False, encoding='utf-8-sig')
print("CSV 저장 완료! 항목 수:", len(df))

CSV 저장 완료! 항목 수: 132


In [ ]:
# 날짜 추출 및 포맷 변경
from datetime import datetime


date_raw = tds[0].get_text(strip=True).split('(')[0].strip()  # "5/3"
# date_raw = tds[0].get_text()

print(date_raw)

In [ ]:
date_obj = datetime.strptime(date_raw, "%m/%d")
race_date = f"2025-{date_obj.month:02}-{date_obj.day:02}"  # 연도는 임의 설정
print(race_date)

In [ ]:
race_data

[]